In [73]:
##intermidiate model training
import os
import subprocess
file_name="MyFile150"
ranking_method="randomForests"
cmd=["java","-jar","RankLib-2.13.jar","-train","uploadedFiles/"+file_name+".txt","-tvs","0.50","-ranker",'8',"-metric2t","NDCG@100","-metric2T","ERR@10","-save","trainedModels/"+'intermidiate_'+ranking_method+".txt","-gmax","9","-norm","zscore"]
if ranking_method=='listNet':
    cmd.append("-epoch")
    cmd.append("6000")
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    # print(x)
    output.append(x)
index=1

In [74]:
##Step1 ranking
import os
import subprocess
file_name="MyFile8"
ranking_method="randomForests"

#Start#generating ranklist for input for given input
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+file_name+".txt","-load","trainedModels/"+ranking_method+".txt","-norm","zscore","-indri","myNewRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
#     print(x)
    output.append(x)
#End#
    
#Start#
score_data={}
label_data = {}
query={}
query_name = {}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
        rem = x.split("=")[1]
        i = rem.find("of")
        rem = rem[i+3:]
        rem = rem[:len(rem)-1]
        query_name[qid] = rem
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))
inputInfo=output[6]
inputInfo=inputInfo[1:len(inputInfo)-1]
modelInfo=output[4]
#End#

#Start#
file1 = open("myNewRankedLists.txt","w")
for qid in score_data.keys():
    cnt = 1
    for doc in score_data[qid]:
        s=str(qid)+" Q0 docid = "+str(doc[2])+"_of_"+query_name[qid]+" "+str(cnt)+" "+str(doc[0])+" indri\n"
        cnt = cnt+1
#         print(s)
        file1.write(s)
file1.close()


In [50]:
s = " 3_of_4_Years_of_FAN\n"
i = s.find("of")
s = s[i+3:]
s = s[:len(s)-1]
print(s)

{1: [[0.80202, 4542028.517692, 32],
  [0.80084, 4531288.518278, 57],
  [0.80039, 3852988.517562, 26],
  [0.80006, 4369348.518529, 69],
  [0.7991, 4357708.51876, 80],
  [0.79903, 3920548.517982, 45],
  [0.79899, 4365088.518781, 81],
  [0.79898, 4520008.519324, 107],
  [0.79889, 3927088.518236, 55],
  [0.79889, 4382308.519242, 103],
  [0.79836, 3876448.518173, 52],
  [0.79796, 4476328.51857, 71],
  [0.79769, 4397548.518802, 82],
  [0.79723, 4365208.518508, 68],
  [0.79703, 4552588.520185, 149],
  [0.79675, 5511148.517757, 35],
  [0.7967, 3965968.518466, 66],
  [0.79648, 4439728.519509, 116],
  [0.79641, 4430848.519489, 115],
  [0.79636, 4441468.518927, 88],
  [0.79634, 4008988.518487, 67],
  [0.79624, 4553848.519407, 111],
  [0.79586, 3866728.517949, 44],
  [0.79558, 5539588.518361, 61],
  [0.79552, 5542648.517213, 10],
  [0.79541, 4517368.519777, 129],
  [0.795, 3964228.518009, 46],
  [0.79481, 4357828.519219, 102],
  [0.79445, 4262188.51874, 79],
  [0.7944, 4530028.519366, 109],
  [0.7

In [75]:
#Step2 ranking(top-n)
#Start#get top-n tweets id
n=150
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
counter=0
top_n={}
for line in lines:
    line_data=line.split(" ")
#     print(line_data[4])
    qid=line_data[0]
    if qid not in top_n:
        top_n[qid]=[]
        counter=0
    if counter<n:
        top_n[qid].append(line_data[4])
    counter=counter+1
#End#

In [76]:
#get details of top-n tweets and make new file for testing into second phase
file1 = open("uploadedFiles/"+file_name+".txt","r")
tweets={}
for x in file1:
    curr_row=x.split(" ")
    l=len(curr_row)
    doc_id=curr_row[l-1]
    doc_id=doc_id[:len(doc_id)-1]
    tweets[doc_id]=x
    
file1 = open("uploadedFiles/intermidiate_"+file_name+".txt","w")
remove_feature_no=13
for key in top_n.keys():
    for doc in top_n[key]:
        doc_info=tweets[doc].split(" ")
        left_part=doc_info[:14]
        right_part=doc_info[15:]
        for i in range(0,len(right_part)):
            parti=right_part[i].split(":")
            if len(parti)==2:
                parti[0]=str(int(parti[0])-1)
            right_part[i]=":".join(parti)
#         print(right_part)
#         print(left_part)
        file1.write(" ".join(left_part)+" "+" ".join(right_part))
file1.close()

In [77]:
#second phase ranking 
import os
import subprocess
intermidiate_file_name="intermidiate_"+file_name
intermidiate_ranking_method="intermidiate_"+ranking_method
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+intermidiate_file_name+".txt","-load","trainedModels/"+intermidiate_ranking_method+".txt","-norm","zscore","-indri","intermidiateRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)


Discard orig. features
Model file: trainedModels/intermidiate_randomForests.txt
Feature normalization: zscore
Model:  Random Forests
\rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]: 0... \rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]... [Done.]            
(11 ranked lists, 1650 entries read)


In [78]:
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
rankList={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList:
        rankList[qid]=[]
    rankList[qid].append(line)
file.close()
file = open("intermidiateRankedLists.txt","r")
lines = file.readlines()
rankList2={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList2:
        rankList2[qid]=[]
    rankList2[qid].append(line)
file.close()

for key in rankList2.keys():
    for i in range(0, len(rankList2[key])):
        rankList[key][i]=rankList2[key][i]
file=open("myNewRankedLists.txt","w")
for key in rankList.keys():
    for i in range(0, len(rankList[key])):
        file.write(rankList[key][i])
file.close()


In [ ]:
#NDCG calculation

In [47]:
score_data={}
label_data = {}
query={}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()

In [ ]:
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))